In [1]:
#=======================================================================================
# sentence-bert와 Faiss 라이브러리를 이용하여 search 하는 예시임
# => Faiss는 facebook에서 만든 vector 유사도를 측정하는 라이브러리로, 
# 기존 numpy 나 scikit-learn 에서 제공하는 cosine similarity 보다 강력하며, GPU도 지원한다.
#
# 참고 
# 번역본 : https://ichi.pro/ko/transformers-mich-faissleul-sayonghayeo-simaentig-geomsaeg-enjin-eul-guchughaneun-bangbeob-242711337083112
# (원본 : https://medium.com/towards-data-science/how-to-build-a-semantic-search-engine-with-transformers-and-faiss-dcbea307a0e8)
#
# 설치
# Faiss는 설치는 https://github.com/facebookresearch/faiss/blob/main/INSTALL.md 참조
# GPU 버전 : $ conda install -c conda-forge faiss-gpu
# 
#=======================================================================================
#!conda install -c conda-forge faiss-gpu #보통 이거로 설치하면 됨

import faiss
import pandas as pd
import numpy as np

from os import sys
sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging

device = GPU_info()
seed_everything(111)
logger = mlogging(loggername="Faiss", logfilename="Faiss")

logfilepath:bwdataset_2022-03-28.log
logfilepath:qnadataset_2022-03-28.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
logfilepath:Faiss_2022-03-28.log


In [2]:
# excel 파일 로딩 (임베딩할 데이터)
file_fpath = "../korpora/감성대화말뭉치/감성대화말뭉치(최종데이터)_Validation.xlsx"
#df = pd.read_csv(file_fpath, encoding='euc-kr')
df = pd.read_excel(file_fpath)

print(len(df))

5130


In [3]:
df[0:2]

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,23093,청년,여성,"진로, 취업, 직장",해당없음,분노,툴툴대는,요즘 부모님과 많이 부딪혀.,어떤 일로 부딪히시나요?,난 하고 싶은 일이 있는데 부모님은 안정적인 일만 하라고 하셔. 내 마음대로 하지도...,부모님과의 충돌로 힘드시겠군요.,응. 그래도 난 내가 하고 싶은 일을 위해 노력할 거야.,자신이 하고 싶은 일을 목표로 하기로 하셨군요.,NaN,NaN
1,32848,청소년,남성,가족관계,해당없음,슬픔,비통한,엄마가 결국 집을 나갔어. 너무 너무 슬퍼.,어머니께서 집을 나가셨다니 정말 슬프셨겠어요. 좀 더 자세히 얘기해주시겠어요?,엄마도 엄마만의 인생이 있을 테니 가지 말라고 할 수가 없었어. 그래서 더 마음이 ...,정말 많이 답답하셨겠어요. 어떻게 하는 게 기분이 좀 풀릴까요?,일단 좀 자야겠어.,일단 휴식을 가질 생각이시군요.,NaN,NaN


In [4]:
# sentence bert 모델 로딩
from sentence_transformers import SentenceTransformer
model_path = "../model/bert/bmc_fpt_kowiki20200920.train-sbert-tsmodel-0310"
smodel = SentenceTransformer(model_path)
smodel.to(device)  # GPU 설정

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [5]:
#인코딩
embeddings = smodel.encode(df.사람문장1.to_list(), show_progress_bar=True, convert_to_tensor=False)

Batches:   0%|          | 0/161 [00:00<?, ?it/s]

In [6]:
# data tyep을 float32 array형으로 변경
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

In [7]:
# instance index 생성
index = faiss.IndexFlatL2(embeddings.shape[1])

In [8]:
# id를 매핑 시켜줌
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, df.번호.values)

In [9]:
# 1번째 데이터 값과 유사한 값 검색 해 봄
distance, idx = index.search(np.array([embeddings[1]]), k=10)
print(distance)
print(idx)

[[ 0.       55.556606 58.96285  59.844555 60.291656 61.13421  67.78666
  70.58383  76.5297   79.211   ]]
[[32848 50821 10667 11181 47854 37672  8973 29562 38994  6666]]


In [10]:
# 10개 검색된 id를 가지고 실제 문장을 출력해 봄
def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.번호 == idx][column]) for idx in I[0]]

id2details(df, idx, '사람문장1')


[['엄마가 결국 집을 나갔어. 너무 너무 슬퍼.'],
 ['아내가 아이들을 데리고 친정으로 돌아가서 슬퍼.'],
 ['그렇게 나를 힘들게 하던 시어머니가 돌아가셨어.'],
 ['요즘 돌아가신 엄마가 자꾸 생각나서 눈물이 나.'],
 ['엄마 때문에 속상해 죽겠어.'],
 ['아내가 나와 싸우고 집을 나갔어. 너무 걱정된다.'],
 ['오늘 아침에 엄마하고 싸웠어. 슬퍼.'],
 ['엄마는 내가 버리라고 한 옷을 자꾸 입어서 짜증 나!'],
 ['잠깐 자리를 비웠는데 아내가 집을 나갔어! 치매가 있는데 어딜 간 건지!'],
 ['우리 엄마를 생각하면 슬퍼.']]

In [11]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

In [13]:
# 실제 한문장 쿼리 해봄
user_query = "나는 오늘 여행갈거라서 너무 행복하다"
distance, idx = vector_search([user_query], smodel, index, num_results=10)

print(distance)
print(idx)
print(id2details(df, idx, '사람문장1'))


[[71.35812  71.45471  85.22177  85.958565 89.20883  90.06891  91.33994
  92.94822  96.01962  97.03589 ]]
[[41280 41333  6101 39155 36631 22566 46456 50072 48906 10756]]
[['나 요즘 너무 기뻐!'], ['요즘 사는 게 즐겁고 행복해.'], ['직장에서 맡은 이번 프로젝트 결과가 정말 만족스러워! 너무 뿌듯해!'], ['완치 후 집에 오니 지인들이 선물이 많이 와있다. 인생 잘 산 것 같아서 기뻐.'], ['내가 열심히 한 만큼 성적이 잘 나와서 행복해.'], ['나 직장 옮겼어. 너무 좋아.'], ['오늘 너무 좋은 소식을 들어서 날아갈 듯이 기뻐!'], ['여행 계획을 짜는데 우리 가족 모두가 내 말을 수용해주니 정말 기뻐.'], ['결혼을 하고 나니 생활이 더 안정감 있는 것 같아서 너무 좋아.'], ['내 육십 평생 살아오면서 마음이 통하는 친구가 있다는 게 정말 만족스러워.']]


In [ ]:
df[df.번호 == 41280]['사람문장1']

In [ ]:
# 여러문장 쿼리 해봄
user_query = ["나는 오늘 여행갈거라서 너무 행복하다", 
             "나는 오늘 너무 슬프다"]

vector = smodel.encode(user_query)
# print(len(vector))
distance, idx = index.search(np.array(vector).astype("float32"), k=10)
print(distance)
print(idx)
print([list(df[df.번호 == num]['사람문장1']) for num in I[0]])
print([list(df[df.번호 == num]['사람문장1']) for num in I[1]])
